In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import pickle
import unicodedata
import re
import numpy as np
import os
import io
import time

#토큰을 단어로 바꾸는 영어토큰 역-딕셔너리
#예: <4> -> "the"
#크기는 3만
with open("eng_reverse_dictionary.bin","rb") as file :
    eng_rdictionary = pickle.load(file)
    
#역딕셔너리의 0번째는 알수 없는 토큰 UNK
print(eng_rdictionary[0])
#역딕셔너리의 1번째는 시계열 시작 토큰 <s>
print(eng_rdictionary[1])
#역딕셔너리의 2번째는 시계열 끝 토큰 </s>
print(eng_rdictionary[2])


print(eng_rdictionary[4])
print(len(eng_rdictionary.keys()))

#토큰을 단어로 바꾸는 한글토큰 역-딕셔너리
#예: <10> -> "의"
with open("kor_reverse_dictionary.bin","rb") as file :
    kor_rdictionary = pickle.load(file)

print(kor_rdictionary[10])
print(len(kor_rdictionary.keys()))

#영어 단어 토큰을 벡터로 바꾸는 임베딩 정보
with open("eng_embedding.bin","rb") as file :
    eng_embedding = pickle.load(file)
print(len(eng_embedding))

#한글 단어 토큰을 벡터로 바꾸는 임베딩 정보
with open("kor_embedding.bin","rb") as file :
    kor_embedding = pickle.load(file)
print(len(kor_embedding))


#토큰 번호로 이루어진 영문 데이터 불러오기
with open("eng_indexed.bin", "rb") as file:
    #maxlen = [-1]
    eng_indexed = []
    #tmp_i = -1
    for i in range(692109):
        tmp_s = pickle.load(file)
        eng_indexed.append(tmp_s)
        #if len(tmp_s) > maxlen[-1] : 
        #    maxlen.append(len(tmp_s))
        #    tmp_i = i
#print(maxlen[-1], tmp_i, eng_indexed[tmp_i])


#토큰 번호로 이루어진 국문 데이터 불러오기
with open("kor_indexed.bin", "rb") as file:
    #maxlen = [-1]
    kor_indexed = []
    #tmp_i = -1
    for i in range(692109):
        tmp_s = pickle.load(file)
        kor_indexed.append(tmp_s)
        #if len(tmp_s) > maxlen[-1] : 
        #    maxlen.append(len(tmp_s))
        #    tmp_i = i
#print(maxlen[-1], tmp_i, kor_indexed[tmp_i])

del tmp_s




eng_rdict_size = len(eng_rdictionary)
kor_rdict_size = len(kor_rdictionary)
embedding_size = len(eng_embedding[0])
en_length = 60
kr_length = 60
sentences_num = len(eng_indexed)
num_epochs = 5
batch_size = 96
num_units = 256
learning_rate = 0.0001
training = True

if not training:
    import nltk
    #nltk.download()
    batch_size = 1
    with open("eng_dictionary.bin","rb") as file :
        eng_dictionary = pickle.load(file)

UNK
<s>
</s>
the
30000
의
30000
30000
30000


In [2]:
def token_to_eng(tokens):
  sentences = []
  for i in range(tokens.shape[0]):
    s = ""
    for j in tokens[i]:
      if j == 2:
        break;
      s += eng_rdictionary[j]+" "
    sentences.append(s)
  return sentences

def token_to_kor(tokens):
  sentences = []
  for i in range(tokens.shape[0]):
    s = ""
    for j in tokens[i]:
      if j == 2:
        break;
      s += kor_rdictionary[j]+" "
    sentences.append(s)
  return sentences

In [3]:
print(eng_indexed[1])
for token in eng_indexed[1]:
  print(token, "-->", eng_rdictionary[token])

print(kor_indexed[1])
for token in kor_indexed[1]:
  print(token, "-->", kor_rdictionary[token])

#inp, targ, tmp_eng_size, tmp_kor_size = batch_generator.make_batch(1, en_length, kr_length)
#enc_emb = eng_embedding
#
#enc_inp = tf.nn.embedding_lookup(enc_emb, inp)
#print(enc_inp)

[4, 93, 841, 15370, 4, 50, 479, 4, 156, 3]
4 --> the
93 --> world
841 --> cup
15370 --> amused
4 --> the
50 --> people
479 --> across
4 --> the
156 --> country
3 --> .
[3116, 11, 1332, 329, 13, 44, 18, 9, 1784, 32, 4, 166, 7, 3]
3116 --> 월드컵
11 --> 은
1332 --> 온
329 --> 나라
13 --> 에
44 --> 사람
18 --> 들
9 --> 을
1784 --> 즐겁
32 --> 게
4 --> 하
166 --> 였
7 --> 다
3 --> .


In [4]:
class BatchGenerator:
    def __init__(self, en, kr, totalnum = 692109, shuffle = True):
        self.totalnum = totalnum
        self.en = en
        self.kr = kr
        self.cursor = 0
        self.mask = np.arange(self.totalnum)
        self.shuffle = shuffle
        if self.shuffle :
            np.random.shuffle(self.mask)
    def make_batch(self, batch_size, en_length = 60, kr_length = 40):
        tmp_eng = np.ndarray((batch_size, en_length), dtype = np.int32)
        tmp_eng_size  = np.ndarray((batch_size), dtype = np.int32) 
        tmp_kor = np.ndarray((batch_size, kr_length), dtype = np.int32)
        tmp_kor_size  = np.ndarray((batch_size), dtype = np.int32)
        tmp_eng_mask = np.zeros([batch_size, en_length], dtype=bool)
        tmp_kor_mask = np.zeros([batch_size, kr_length], dtype=bool)
        for i in range(batch_size):
            tmps = self.en[self.mask[self.cursor]]
            if len(tmps) >= en_length :
                tmp_eng_size[i] = en_length
                tmps = tmps[:en_length]
            if len(tmps) < en_length :
                tmp_eng_size[i] = len(tmps)
                tmps = tmps + ([2] * (en_length-len(tmps)))
            tmp_eng[i] = np.array(tmps)
            tmps = self.kr[self.mask[self.cursor]]
            if len(tmps) >= kr_length :
                tmp_kor_size[i] = kr_length
                tmps = tmps[:kr_length-1] + [2]
            if len(tmps) < kr_length :
                tmp_kor_size[i] = len(tmps) + 1
                tmps = tmps + [2] + ([1] * (kr_length-len(tmps)-1))
            tmp_kor[i] = np.array(tmps)
            self.cursor += 1
            if self.cursor >= self.totalnum :
                self.cursor = 0
                if self.shuffle :
                    np.random.shuffle(self.mask)
            tmp_eng_mask[i, :tmp_eng_size[i]] = 1
            tmp_kor_mask[i, :tmp_kor_size[i]] = 1
        tmp_eng = np.array(tmp_eng, dtype=np.int32)
        tmp_kor = np.array(tmp_kor, dtype=np.int32)
        

        
        return tmp_eng, tmp_kor, tmp_eng_size, tmp_kor_size, tmp_eng_mask, tmp_kor_mask



"""
kor_indexed = []
i = 0
for _ in range(692109):
    i = 8 + (i * 5 % 87)
    kor_indexed.append(list(range(i)))
#print(kor_indexed[0:100])
"""

batch_generator = BatchGenerator(eng_indexed, kor_indexed, totalnum = sentences_num, shuffle = False)

tmp_eng, tmp_kor, tmp_eng_size, tmp_kor_size, em, km = batch_generator.make_batch(batch_size, en_length, kr_length)
print(tmp_eng[0:5])
print(tmp_eng_size[0:5])
print(tmp_kor[0:5])
print(tmp_kor_size[0:5])
for i in range(5):
    print(eng_indexed[i])
#print(tmp_eng.shape)
print(em[0:5])
print(km[0:5])

[[    0   467   608    11  1677    49     4   213    99  5023   100     7
      4   102     3     2     2     2     2     2     2     2     2     2
      2     2     2     2     2     2     2     2     2     2     2     2
      2     2     2     2     2     2     2     2     2     2     2     2
      2     2     2     2     2     2     2     2     2     2     2     2]
 [    4    93   841 15370     4    50   479     4   156     3     2     2
      2     2     2     2     2     2     2     2     2     2     2     2
      2     2     2     2     2     2     2     2     2     2     2     2
      2     2     2     2     2     2     2     2     2     2     2     2
      2     2     2     2     2     2     2     2     2     2     2     2]
 [    4   579    11     0   213     4  4644     7     4   443     3     2
      2     2     2     2     2     2     2     2     2     2     2     2
      2     2     2     2     2     2     2     2     2     2     2     2
      2     2     2     2     2     

In [5]:
class Encoder(tf.keras.Model):
  def __init__(self, enc_units):
    super(Encoder, self).__init__()
    self.enc_units = enc_units
    #우리 예제에서는, 별도의 토큰 임베딩을 이용한다.
    #self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.rnn = tf.keras.layers.RNN(
            tf.keras.layers.LSTMCell(self.enc_units, activation = 'relu'),
            return_sequences = True, 
            return_state = True)

  def call(self, x, hidden, mask):
    #x = self.embedding(x)
    if mask is not None:
      mask = tf.convert_to_tensor(mask)
    output, *state = self.rnn(x, mask=mask, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self, batch_sz):
    return [tf.zeros((batch_sz, self.enc_units)), tf.zeros((batch_sz, self.enc_units))]


class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values, value_mask):
    # 쿼리 은닉 상태(query hidden state)는 (batch_size, hidden size)쌍으로 이루어져 있습니다.
    # query_with_time_axis은 (batch_size, 1, hidden size)쌍으로 이루어져 있습니다.
    # values는 (batch_size, max_len, hidden size)쌍으로 이루어져 있습니다.
    # 스코어(score)계산을 위해 덧셈을 수행하고자 시간 축을 확장하여 아래의 과정을 수행합니다.
    #query_with_time_axis = tf.expand_dims(query, 1)
    query_with_time_axis = query
    # score는 (batch_size, max_length, 1)쌍으로 이루어져 있습니다.
    # score를 self.V에 적용하기 때문에 마지막 축에 1을 얻습니다.
    # self.V에 적용하기 전에 텐서는 (batch_size, max_length, units)쌍으로 이루어져 있습니다.
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))
    
    score += -1e5*tf.cast(tf.expand_dims(~value_mask, -1), dtype=score.dtype)
    
    # attention_weights는 (batch_size, max_length, 1)쌍으로 이루어져 있습니다. 
    attention_weights = tf.nn.softmax(score, axis=1)
    
    # 덧셈이후 컨텍스트 벡터(context_vector)는 (batch_size, hidden_size)쌍으로 이루어져 있습니다.
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights
  
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, dec_units):
    super(Decoder, self).__init__()
    self.dec_units = dec_units
    #우리 예제에서는, 별도의 토큰 임베딩을 이용한다.
    #self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.rnn = tf.keras.layers.RNN(
            tf.keras.layers.LSTMCell(self.dec_units, activation = 'relu'),
            return_sequences = True, 
            return_state = True)
    
    self.fc = tf.keras.layers.Dense(vocab_size)

    # 어텐션을 사용합니다.
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output, enc_mask):
    

    # 임베딩층을 통과한 후 x는 (batch_size, 1, embedding_dim)쌍으로 이루어져 있습니다.
    #x = self.embedding(x)

    # 컨텍스트 벡터와 임베딩 결과를 결합한 이후 x의 형태는 (batch_size, 1, embedding_dim + hidden_size)쌍으로 이루어져 있습니다.
    #x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # 위에서 결합된 벡터를 LSTM에 전달합니다.
    output, *state = self.rnn(x)
    
    # enc_output는 (batch_size, max_length, hidden_size)쌍으로 이루어져 있습니다.
    context_vector, attention_weights = self.attention(output, enc_output, enc_mask)
    #print(context_vector.shape)
    # 컨텍스트 벡터와 디코더 출력 결과를 결합한 이후 x의 형태는 (batch_size, 1, hidden_size * 2)쌍으로 이루어져 있습니다.
    output = tf.concat([tf.expand_dims(context_vector, 1), output], axis=-1)
    
    # output은 (batch_size * 1, hidden_size)쌍으로 이루어져 있습니다.
    output = tf.reshape(output, (-1, output.shape[2]))

    # output은 (batch_size, vocab)쌍으로 이루어져 있습니다.
    x = self.fc(output)

    return x, state, attention_weights
  


In [6]:
encoder = Encoder(num_units)
decoder = Decoder(eng_rdict_size, num_units)



optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred, mask):
  ##mask = tf.math.logical_not(tf.math.equal(real, 1))
  ##
  #mask = tf.constant(mask)
  loss_ = loss_object(real, pred)
  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)



checkpoint_dir = './test_training_checkpoints3'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)


checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:

N_CANDIDATE = 256


@tf.function
def train_step(inp, targ, enc_emb, dec_emb, enc_mask, dec_mask, BATCH_SIZE,):
  loss = 0
  enc_hidden = encoder.initialize_hidden_state(BATCH_SIZE)
  with tf.GradientTape() as tape:
    enc_inp = tf.nn.embedding_lookup(enc_emb, inp)
    enc_output, enc_hidden = encoder(enc_inp, enc_hidden, enc_mask)
    
    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([1] * BATCH_SIZE, 1)
    candidate_idx = np.arange(eng_rdict_size)
    np.random.shuffle(candidate_idx)
    candidate_mask = np.zeros([1, eng_rdict_size])
    candidate_mask[0, candidate_idx[:N_CANDIDATE]] = 1.0
    predict_mask = tf.one_hot(tf.reshape(targ, [-1]), eng_rdict_size)
    predict_mask = tf.math.reduce_max(predict_mask, axis = 0, keepdims=True)
    predict_mask = tf.math.maximum(predict_mask, candidate_mask)
    
    # 교사 강요(teacher forcing) - 다음 입력으로 타겟을 피딩(feeding)합니다.
    for t in range(0, targ.shape[1]):
      dec_input = tf.nn.embedding_lookup(dec_emb, dec_input)
      # enc_output를 디코더에 전달합니다.
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output, enc_mask)
      
      predictions *= predict_mask
      
      #print(predictions[0, :10])
      #assert(False)
      loss += loss_function(targ[:, t], predictions, dec_mask[:, t])

      # 교사 강요(teacher forcing)를 사용합니다.
      dec_input = tf.expand_dims(targ[:, t], 1)
      

  batch_loss = (loss / tf.reduce_sum( tf.cast(targ != 1, dtype = tf.float32) ) )

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss


def evaluate(inp, enc_emb, dec_emb, enc_mask, dec_mask, BATCH_SIZE):

  enc_inp = tf.nn.embedding_lookup(enc_emb, inp)
  enc_hidden = encoder.initialize_hidden_state(BATCH_SIZE)
  enc_output, enc_hidden = encoder(enc_inp, enc_hidden, enc_mask)

  dec_hidden = enc_hidden

  dec_input = tf.expand_dims([1] * BATCH_SIZE, 1)

  outputs = []
  attention_weights = []
  # 교사 강요(teacher forcing) - 다음 입력으로 타겟을 피딩(feeding)합니다.
  for t in range(kr_length):
    dec_input = tf.nn.embedding_lookup(dec_emb, dec_input)
    # enc_output를 디코더에 전달합니다.
    predictions, dec_hidden, attention_weight = decoder(dec_input, dec_hidden, enc_output, enc_mask)

    attention_weights.append(attention_weight)
    #loss += loss_function(targ[:, t], predictions)

    # 교사 강요(teacher forcing)를 사용합니다.
    dec_input = tf.argmax(predictions, axis=-1)
    outputs.append(dec_input)
    dec_input = tf.expand_dims(dec_input, 1)
    
  outputs = np.stack(outputs, axis=1)
  attention_weights = np.stack(attention_weights, axis=1)
  return outputs, attention_weights


EPOCHS = 100

batch_generator = BatchGenerator(eng_indexed, kor_indexed, totalnum = sentences_num, shuffle = False)

#tmp_eng, tmp_kor, tmp_eng_size, tmp_kor_size = batch_generator.make_batch(batch_size, en_length, kr_length)
steps_per_epoch = sentences_num // batch_size + 1

#prepare early stopping
best_loss = 1000
tol = 9999
miss_count = 0
#annealing_count = 4

for epoch in range(EPOCHS):
  start = time.time()

  total_loss = 0

  
  for batch in range(steps_per_epoch):
    inp, targ, _, _, eng_mask, kor_mask= batch_generator.make_batch(batch_size, en_length, kr_length)
    
    #print(eng_mask)
    batch_loss = train_step(inp, targ, eng_embedding, kor_embedding, eng_mask, kor_mask, batch_size)
    
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  outputs , attention_weights = evaluate(inp[:3], eng_embedding, kor_embedding, eng_mask[:3], kor_mask[:3], 3)
  
  inp_sentences = token_to_eng(inp[:3])
  targ_sentences = token_to_kor(targ[:3])
  output_sentences = token_to_kor(outputs[:3])
  
  for i in range(3):
    print("input:", inp_sentences[i])
    print("targ:", targ_sentences[i])
    print("output:", output_sentences[i])
  
  
  
  # 에포크가 2번 실행될때마다 모델 저장 (체크포인트)
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)
  
  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
  if total_loss / steps_per_epoch < best_loss:
    best_loss = total_loss / steps_per_epoch
    miss_count = 0
  else:
    miss_count += 1
    if miss_count == tol:
      print("early stopping activated...")
      break
    
      
  


Epoch 1 Batch 0 Loss 0.1074
Epoch 1 Batch 100 Loss 0.0885
Epoch 1 Batch 200 Loss 0.0981
Epoch 1 Batch 300 Loss 0.0976
Epoch 1 Batch 400 Loss 0.1031
Epoch 1 Batch 500 Loss 0.1031
Epoch 1 Batch 600 Loss 0.1029
Epoch 1 Batch 700 Loss 0.1020
Epoch 1 Batch 800 Loss 0.1018
Epoch 1 Batch 900 Loss 19.3271
Epoch 1 Batch 1000 Loss 0.1007
Epoch 1 Batch 1100 Loss 0.1003
Epoch 1 Batch 1200 Loss 0.0997
Epoch 1 Batch 1300 Loss 0.0994
Epoch 1 Batch 1400 Loss 0.1180
Epoch 1 Batch 1500 Loss 0.0985
Epoch 1 Batch 1600 Loss 2035.6350
Epoch 1 Batch 1700 Loss 0.5653
Epoch 1 Batch 1800 Loss 0.0981
Epoch 1 Batch 1900 Loss 0.0952
Epoch 1 Batch 2000 Loss 0.0951
Epoch 1 Batch 2100 Loss 0.0950
Epoch 1 Batch 2200 Loss 0.0928
Epoch 1 Batch 2300 Loss 0.0939
Epoch 1 Batch 2400 Loss 0.1502
Epoch 1 Batch 2500 Loss 0.0934
Epoch 1 Batch 2600 Loss 0.0930
Epoch 1 Batch 2700 Loss 5.6364
Epoch 1 Batch 2800 Loss 10062.0166
Epoch 1 Batch 2900 Loss 0.4810
Epoch 1 Batch 3000 Loss 0.1028
Epoch 1 Batch 3100 Loss 0.0890
Epoch 1 Batc

Epoch 4 Batch 0 Loss 0.0545
Epoch 4 Batch 100 Loss 0.0559
Epoch 4 Batch 200 Loss 0.0544
Epoch 4 Batch 300 Loss 0.0547
Epoch 4 Batch 400 Loss 0.0525
Epoch 4 Batch 500 Loss 0.0560
Epoch 4 Batch 600 Loss 0.0560
Epoch 4 Batch 700 Loss 0.0530
Epoch 4 Batch 800 Loss 0.0553
Epoch 4 Batch 900 Loss 0.0577
Epoch 4 Batch 1000 Loss 0.0541
Epoch 4 Batch 1100 Loss 0.0543
Epoch 4 Batch 1200 Loss 0.0548
Epoch 4 Batch 1300 Loss 0.0538
Epoch 4 Batch 1400 Loss 0.0526
Epoch 4 Batch 1500 Loss 0.0541
Epoch 4 Batch 1600 Loss 0.0550
Epoch 4 Batch 1700 Loss 0.0540
Epoch 4 Batch 1800 Loss 0.0534
Epoch 4 Batch 1900 Loss 0.0535
Epoch 4 Batch 2000 Loss 0.0558
Epoch 4 Batch 2100 Loss 0.0514
Epoch 4 Batch 2200 Loss 0.0553
Epoch 4 Batch 2300 Loss 0.0800
Epoch 4 Batch 2400 Loss 0.0532
Epoch 4 Batch 2500 Loss 0.0552
Epoch 4 Batch 2600 Loss 0.0519
Epoch 4 Batch 2700 Loss 0.0549
Epoch 4 Batch 2800 Loss 0.0512
Epoch 4 Batch 2900 Loss 0.0672
Epoch 4 Batch 3000 Loss 0.0536
Epoch 4 Batch 3100 Loss 0.0669
Epoch 4 Batch 3200 L

Epoch 7 Batch 1000 Loss 0.0415
Epoch 7 Batch 1100 Loss 0.0438
Epoch 7 Batch 1200 Loss 0.0404
Epoch 7 Batch 1300 Loss 0.0391
Epoch 7 Batch 1400 Loss 0.0436
Epoch 7 Batch 1500 Loss 0.0427
Epoch 7 Batch 1600 Loss 0.0384
Epoch 7 Batch 1700 Loss 0.0376
Epoch 7 Batch 1800 Loss 0.0401
Epoch 7 Batch 1900 Loss 0.0422
Epoch 7 Batch 2000 Loss 0.0399
Epoch 7 Batch 2100 Loss 0.0405
Epoch 7 Batch 2200 Loss 0.0422
Epoch 7 Batch 2300 Loss 0.0395


KeyboardInterrupt: 

In [8]:
outputs , attention_weights = evaluate(inp[:5], eng_embedding, kor_embedding, eng_mask[:5], kor_mask[:5], 5)

inp_sentences = token_to_eng(inp[:5])
targ_sentences = token_to_kor(targ[:5])
output_sentences = token_to_kor(outputs[:5])

for i in range(5):
  print("input:", inp_sentences[i])
  print("targ:", targ_sentences[i])
  print("output:", output_sentences[i])

input: i 'll be back after canceling my appointment . 
targ: 약속 을 취소 하 고 다시 오 ㄹ게 . 
output: 는 것 이 다 . 
input: i think she has a doctor 's appointment . 
targ: 그녀 는 의사 와 진찰 약속 이 있 는 것 같 습니다 . 
output: 는 것 같 아요 . 
input: he is sure to be appointed . or his appointment is certain . 
targ: 그 가 임명 되 는 것 은 확실 하 다 
output: 는 것 이 다 . 
input: i just need to pack a towel . 
targ: 수건 하나 만 싸 면 되 어요 . 
output: 는 것 이 필요 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 하 어야 
input: i smoke about a pack of cigarettes a day . 
targ: 전 하루 에 한 갑 정도 피우 ㅂ니다 . 
output: 는 하루 에 관하 ㄴ 하루 에 관하 ㄴ 하루 에 관하 ㄴ 하루 에 관하 ㄴ 하루 에 관하 ㄴ 하루 에 관하 ㄴ 하루 에 관하 ㄴ 하루 에 관하 ㄴ 하루 에 관하 ㄴ 하루 에 관하 ㄴ 하루 에 관하 ㄴ 하루 에 관하 ㄴ 하루 에 관하 ㄴ 하루 에 관하 ㄴ 하루 에 관하 


In [11]:
encoder.initialize_hidden_state(3)

[<tf.Tensor: shape=(128, 128), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(128, 128), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>]

In [45]:
def evaluate(inp, enc_emb, dec_emb, BATCH_SIZE):

  enc_inp = tf.nn.embedding_lookup(enc_emb, inp)
  enc_hidden = encoder.initialize_hidden_state()
  enc_output, enc_hidden = encoder(enc_inp, enc_hidden)

  dec_hidden = enc_hidden

  dec_input = tf.expand_dims([1] * BATCH_SIZE, 1)

  outputs = []
  attention_weights = []
  # 교사 강요(teacher forcing) - 다음 입력으로 타겟을 피딩(feeding)합니다.
  for t in range(kr_length):
    dec_input = tf.nn.embedding_lookup(dec_emb, dec_input)
    # enc_output를 디코더에 전달합니다.
    predictions, dec_hidden, attention_weight = decoder(dec_input, dec_hidden, enc_output)

    attention_weights.append(attention_weight)
    #loss += loss_function(targ[:, t], predictions)

    # 교사 강요(teacher forcing)를 사용합니다.
    dec_input = tf.argmax(predictions, axis=-1)
    outputs.append(dec_input)
    dec_input = tf.expand_dims(dec_input, 1)
    
  outputs = np.stack(outputs, axis=1)
  attention_weights = np.stack(attention_weights, axis=1)
  return outputs, attention_weights
      

inp, targ, _, _ = batch_generator.make_batch(batch_size, en_length, kr_length)
  
outputs, attention_weights = evaluate(inp, eng_embedding, kor_embedding, batch_size)

print(outputs.shape)
print(attention_weights.shape)
print(outputs[0])



kor_outputs = token_to_kor(outputs)
kor_targ = token_to_kor(targ)
print(kor_targ[0])
print(kor_outputs[0])


(16, 60)
(16, 60, 60, 1)
[ 11  19 106   6   8  24  11 106   9   4  22  34  40 239  29   3  42   0
  43  11 106   9   4  22  34  40 239  29   3  42   0  43  11 106   9   4
  22  34  40 239  29   3  42   0  43  11 106   9   4  22  34  40 239  29
   3  42   0  43  11 106]
이 냄새 는 시간 이 좀 지나 면 없 어 지 ㄹ 것 이 ㅂ니다 . 
은 그 시간 이 ㄴ 것 은 시간 을 하 ㄹ 수 도 모르 ㄴ다 . ( UNK ) 은 시간 을 하 ㄹ 수 도 모르 ㄴ다 . ( UNK ) 은 시간 을 하 ㄹ 수 도 모르 ㄴ다 . ( UNK ) 은 시간 을 하 ㄹ 수 도 모르 ㄴ다 . ( UNK ) 은 시간 
